## Data partitioning

In this notebook we will take the conditioned data that we have, the data should be in the form of a two column csv that has the block numvber on the left and the MEV quantity sorted in to none = 0, low = 1, medium = 2, and high =3. This notebook is specifically for the data with the gas fees and the price volatility of etherium at the time. 

In [5]:
import numpy as np 
import pandas as pd
from numpy import genfromtxt
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json
import csv

In [6]:
#let pull the data in and so that we can check how big the files are. 

my_data = genfromtxt('../Ethdata/randomdatawgv.csv', delimiter=',')
print(my_data)
print(np.shape(my_data))


[[0.000e+00 2.000e+00 1.000e+00 0.000e+00]
 [1.000e+00 1.000e+00 2.000e+00 2.000e+00]
 [2.000e+00 3.000e+00 3.000e+00 1.000e+00]
 ...
 [9.997e+03 0.000e+00 1.000e+00 3.000e+00]
 [9.998e+03 1.000e+00 1.000e+00 3.000e+00]
 [9.999e+03 0.000e+00 3.000e+00 0.000e+00]]
(10000, 4)


In [7]:
# This cell will make what I am refering to as datalets, these are strings of n periods, infact this will create all of the strings of n periods from the data available
# we will use these to train tha models. 

#start by making a dictionary to hold all this data

datalets = {}

# Then we loop over, the default is set from 2-10, this is the length of hte data length, the x vector will be one less than this. 

for i in range(2,10):
    
    # Start with an all zeros np array so we can get started, it will have i columns because that is the number of 
    
    datalet = np.zeros((np.shape(my_data)[0],i*3))
    
    for j in range(i):
        data = np.delete(my_data[:,1],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(int)
        datalet[:,j] = data

        data = np.delete(my_data[:,2],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(float)
        datalet[:,i+j] = data        
        
        data = np.delete(my_data[:,3],range(j),0)
        data = np.append(data,range(j))
        data = data.astype(float)
        datalet[:,2*i+j] = data        
               
    datalet = datalet.astype(float)
    datalet = np.delete(datalet,range(i*(-1),0),0)
        
    datalets["datalet{}".format(i)] = datalet

print(datalets)  

{'datalet2': array([[2., 1., 1., 2., 0., 2.],
       [1., 3., 2., 3., 2., 1.],
       [3., 1., 3., 2., 1., 3.],
       ...,
       [0., 3., 3., 2., 1., 0.],
       [3., 0., 2., 1., 0., 3.],
       [0., 1., 1., 1., 3., 3.]]), 'datalet3': array([[2., 1., 3., ..., 0., 2., 1.],
       [1., 3., 1., ..., 2., 1., 3.],
       [3., 1., 3., ..., 1., 3., 1.],
       ...,
       [2., 0., 3., ..., 2., 1., 0.],
       [0., 3., 0., ..., 1., 0., 3.],
       [3., 0., 1., ..., 0., 3., 3.]]), 'datalet4': array([[2., 1., 3., ..., 2., 1., 3.],
       [1., 3., 1., ..., 1., 3., 1.],
       [3., 1., 3., ..., 3., 1., 1.],
       ...,
       [2., 2., 0., ..., 2., 1., 0.],
       [2., 0., 3., ..., 1., 0., 3.],
       [0., 3., 0., ..., 0., 3., 3.]]), 'datalet5': array([[2., 1., 3., ..., 1., 3., 1.],
       [1., 3., 1., ..., 3., 1., 1.],
       [3., 1., 3., ..., 1., 1., 3.],
       ...,
       [0., 2., 2., ..., 2., 1., 0.],
       [2., 2., 0., ..., 1., 0., 3.],
       [2., 0., 3., ..., 0., 3., 3.]]), 'datalet6': a

## Saving these datalets as CSVs

In [8]:
for x,y in datalets.items():
    headers = []
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('Mev_period{}'.format(i))
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('gas_fees_period{}'.format(i))
    for i in range(int(np.shape(y)[1]/3)):
        headers.append('price_volitility_period{}'.format(i))
    headerstr = ','.join(map(str,headers))
    np.savetxt("..\dateletswgv\{}.csv".format(x), y, delimiter=",", header= headerstr, comments = '')